# Subscriptions

React to changes: validation, logging, computed properties, synchronization.

In [ ]:
# Setup: Install genro-bag (run this cell first on Colab)\n!pip install -q genro-bag

In [ ]:
from genro_bag import Bag

## The Core Idea

Subscribe to changes at any path in the tree. When values change, your callback is invoked.

In [ ]:
bag = Bag()
events = []

def on_change(node, evt, **kw):
    """Called when any change occurs."""
    events.append(f"{evt}: {node.label}")

# Subscribe to all events
bag.subscribe('logger', any=on_change)

# Make changes
bag['name'] = 'Alice'
bag['age'] = 30
bag['name'] = 'Bob'  # Update
del bag['age']  # Delete

print("Events captured:")
for e in events:
    print(f"  {e}")

## Event Types

Subscribe to specific event types: `ins` (insert), `upd` (update), `del` (delete).

In [ ]:
bag = Bag()
inserts = []
updates = []

def on_insert(node, **kw):
    inserts.append(node.label)

def on_update(node, **kw):
    updates.append(node.label)

# Subscribe to specific events
bag.subscribe('tracker', ins=on_insert, upd=on_update)

bag['x'] = 1  # Insert
bag['y'] = 2  # Insert
bag['x'] = 10  # Update
bag['y'] = 20  # Update

print(f"Inserts: {inserts}")
print(f"Updates: {updates}")

## Path-Scoped Subscriptions

Subscribe to changes only within a specific subtree.

In [ ]:
bag = Bag()
db_changes = []

def on_db_change(node, evt, **kw):
    db_changes.append(f"{evt}: {node.label} = {node.value}")

# Only watch the 'config.database' subtree
bag.subscribe('db_watcher', any=on_db_change, path='config.database')

# These trigger the subscription
bag['config.database.host'] = 'localhost'
bag['config.database.port'] = 5432

# This does NOT trigger (different path)
bag['config.logging.level'] = 'DEBUG'

print("Database changes captured:")
for c in db_changes:
    print(f"  {c}")

## Validation Example

Use subscriptions to validate data as it's added.

In [ ]:
bag = Bag()
validation_errors = []

def validate_port(node, **kw):
    """Ensure port is in valid range."""
    if node.label == 'port':
        value = node.value
        if not (1 <= value <= 65535):
            validation_errors.append(f"Invalid port: {value}")

bag.subscribe('validator', ins=validate_port, upd=validate_port)

bag['server.port'] = 8080  # Valid
bag['server.port'] = 99999  # Invalid
bag['server.port'] = -1  # Invalid

print(f"Validation errors: {validation_errors}")

## Computed Properties

Automatically update derived values when dependencies change.

In [ ]:
bag = Bag()

def update_full_name(node, **kw):
    """Recompute full_name when first or last name changes."""
    if node.label in ('first', 'last'):
        first = bag.get_item('user.first', default='')
        last = bag.get_item('user.last', default='')
        if first or last:
            bag['user.full_name'] = f"{first} {last}".strip()

bag.subscribe('computed', ins=update_full_name, upd=update_full_name, path='user')

bag['user.first'] = 'Alice'
print(f"After first: {bag.get_item('user.full_name', default='(not set)')}")

bag['user.last'] = 'Smith'
print(f"After last: {bag['user.full_name']}")

bag['user.first'] = 'Bob'
print(f"After change: {bag['user.full_name']}")

## Unsubscribing

Remove subscriptions when no longer needed.

In [ ]:
bag = Bag()
events = []

bag.subscribe('temp', any=lambda n, **kw: events.append(n.label))

bag['a'] = 1
print(f"Before unsubscribe: {events}")

bag.unsubscribe('temp')

bag['b'] = 2
print(f"After unsubscribe: {events}")

## Key Takeaways

- **Event types**: `ins`, `upd`, `del`, or `any` for all
- **Path scoping**: Watch specific subtrees with `path=`
- **Use cases**: Validation, logging, computed properties, sync
- **Lifecycle**: `subscribe()` to add, `unsubscribe()` to remove